# Proximal Policy Optimization

### Import the libraries

In [62]:
# Calculations
import numpy as np
from numpy import random, sqrt

# Network libraries
import torch
import torch.nn.functional as F, torch.nn as nn, torch.optim as optim
from torch.distributions import Categorical as Categorical

# Game library
from TicTacToe import initialState, randomState, printState, drawState, move, possibleMoves, possibleMovesMask, isOver, gameScore

### Generalized Advantage Estimate (GAE)

The GAE is calculated using the formula:

$ \begin{equation*}
\hat{A_t}^{GAE(\gamma, \lambda)} = (1-\lambda)\left( \hat{A}^{(1)}_t + \lambda\hat{A}^{(2)}_t + \lambda^2\hat{A}^{(3)}_t + ...\right) = \sum_{l=0}^{\infty}(\gamma\lambda)^l\delta^V_{t+l}
\end{equation*}$

where $\delta^V_{t}$ is the TD residual defined by:

$ \begin{equation*}
\delta^V_{t} = r_t + \gamma V^\pi(s_{t+1})-V^\pi(s_t)
\end{equation*}
$



Note that by the definition, we can easily evaulate $ \hat{A_t}^{GAE(\gamma, \lambda)} $ based on $ \hat{A_{t+1}}^{GAE(\gamma, \lambda)} $:

\begin{equation*}
\hat{A_t}^{GAE(\gamma, \lambda)} = \delta^V_t + \gamma\lambda \hat{A}_{t+1}^{GAE(\gamma, \lambda)} 
\end{equation*}


In [63]:
def gae(values, rewards, lmbda = 0.95, gamma = 0.99):
	# Initialize the advangate
	adv = 0
	# Initialize the results
	results = []

	# Starting with the last action, iterate over actions and calculate the advantages
	for t in reversed(range(len(rewards))):
		# Calculate the current value of delta
		delta = rewards[t] + gamma * values[t+1] - values[t]
		# Update the current advantage
		adv = delta + gamma * lmbda * adv
		# Add the estimates advantage to the value, to obtain a predicition of the real value
		results.append(adv+values[t])
	
	# Reverse the results to obtain the right order
	results.reverse()

	# Return the results
	return results

### Actor-Critic Network

In [76]:
class ActorCritic(nn.Module):

	# Initialize the actor and critic
	def __init__(self):
		
		super(ActorCritic, self).__init__()

		# Actor
		self.actor = nn.Sequential(
			nn.Conv2d(1, 8, 2),
			nn.ReLU(),
			nn.Conv2d(8, 32, 2),
			nn.ReLU(),
			nn.Flatten(),
			nn.Linear(32, 9)
		)

		# Critic
		self.critic = nn.Sequential(
			nn.Conv2d(1, 8, 2),
			nn.ReLU(),
			nn.Conv2d(8, 32, 2),
			nn.ReLU(),
			nn.Flatten(),
			nn.Linear(32, 1)
		)

		self.softmax = nn.Softmax(dim=-1)

	
	# Instead of using a forward function, we will split it into 
	# one action for the actor, and one for the critic
	def forward(self):
		raise NotImplementedError
	

	# For a given state, return an action according to a current policy and the log_prob
	# of performing this action
	def act(self, state):
		# Change the state into suitable format
		tensor_state = torch.reshape(state, (-1,1,3,3))#torch.from_numpy(state.reshape(-1, 1, 3,3)).float()

		# Calculate the initial output of the actor
		probs = self.actor(tensor_state)
		
		# Apply the invalid action masking
		probs = torch.where(possibleMovesMask(state, 1), probs, torch.tensor([-1e8]*9))

		# Apply the softmax function to obtain probabilities
		probs = self.softmax(probs)

		# Create a distribution
		dist = Categorical(probs)

		# Pick an action using the generated distribution
		action = dist.sample()

		# Evaluate the log_prob of the chosen action
		log_prob = dist.log_prob(action)

		# Return the action and the log_probabilties
		return action.detach(), log_prob.detach()


	# For a given state and action, return:
	# 	- 	the log_probabilities of the action, according to the current policy, 
	# 	-	the estimated value of the state, according to the critic,
	#	-	the entropy of the distribution
	def evaluate(self, state, action):
		# Change the state into suitable format
		tensor_state = torch.reshape(state, (-1,1,3,3))#torch.from_numpy(state.reshape(-1, 1, 3,3)).float()

		# Calculate the initial output of the actor
		probs = self.actor(tensor_state)
		
		# Apply the invalid action masking
		probs = torch.where(possibleMovesMask(state, 1), probs, torch.tensor([-1e8]*9))

		# Apply the softmax function to obtain probabilities
		probs = self.softmax(probs)

		# Create a distribution
		dist = Categorical(probs)

		# Evaluate the log_prob of the chosen action
		log_prob = dist.log_prob(action)

		# Calculate the entropy of the distribution
		dist_entropy = dist.entropy()

		# Calculate the evaluation of the position
		state_eval = self.critic(tensor_state)

		# Return the action and the log_probabilties
		return log_prob, state_eval, dist_entropy

### Hyperparameters

In [77]:
# Learning rate
learning_rate = 0.001

# Objectiive function parameters
c_1 = 0.5
c_2 = 0.01

### Model declaration

In [78]:
# Network model
model = ActorCritic()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
# MSE Loss
mse_loss = nn.MSELoss()

### Data preparation

In [79]:
# For given states, actions, log_probabilities of actions, advantages and predicted values,
# prepare batches of a given size
def generate_batches(states, actions, log_probs, advantages, pred_values, batch_size):
	# Evaluate the number of batches that we can create
	num_batches = (states.size(0))//batch_size
	# Create a permutation of indices
	perm = np.array(range(len(states)))
	np.random.shuffle(perm)

	# Based on the permutation, prepare and yield batches
	for i in range(num_batches):
		_range = perm[i*batch_size:(i+1)*batch_size]
		yield states[_range], actions[_range], log_probs[_range], advantages[_range], pred_values[_range]

### PPO Step

PPO aims to maximize the following objective function:

\begin{equation*}
L_t^{CLIP+VF+S}(\theta) = \hat{\mathbb{E}}_t \left[ L_t^{CLIP}(\theta) - c_1 L_t^{VF}(\theta) + c_2 S[\pi_\theta](s_t) \right]
\end{equation*}

where:
\begin{equation*}
L_t^{CLIP}(\theta) = \hat{\mathbb{E}}_t \left[ \text{min}\left(r_t(\theta)\hat{A}_t\text{, clip}\left( r_t(\theta), 1-\epsilon, 1+\epsilon \right)\hat{A}_t\right) \right]
\end{equation*}

\begin{equation*}
L_t^{VF}(\theta) = \left( V_\theta(s_t) - V_t^{targ} \right)^2
\end{equation*}


$c_1$ and $c_2$ are hyperparameters, $S[\pi](s_t)$ is an entropy bonus and the ratio $r_t(\theta)$ is defined as:

\begin{equation*}
r_t(\theta) = \frac{\pi_\theta(a_t | s_t)}{\pi_{\theta_{old}}(a_t|s_t)} = \text{exp}\left(\text{log }\pi_\theta(a_t | s_t) - \text{ log }\pi_{\theta_{old}}(a_t|s_t)\right)
\end{equation*}

In [94]:
# Perform the PPO step for the given number of epochs and data
def ppo_step(epochs, batch_size, states, actions, log_probs, advantages, pred_values, epsilon = 0.2, c_1 = 0.5, c_2 = 0.001):
	# In every epoch
	for _ in range(epochs):
		# Iterate over generated batches:
		for _states, _actions, _log_probs, _advantages, _values in generate_batches(states, actions, log_probs, advantages, pred_values, batch_size):
			# Calculate the new log_probabilities, evaluation and entropy
			new_log_prob, state_eval, dist_entropy = model.evaluate(_states, _actions)

			# Evaluate the value function loss
			l_vf = mse_loss(state_eval, _values).mean()

			# Evaluate the clipped actor loss
			ratio = (new_log_prob-_log_probs).exp()
			l_clip = torch.min(ratio*_advantages, torch.clamp(ratio, 1-epsilon, 1+epsilon)*_advantages).mean()
			
			# Evaluate the objective function (note that we need to swap signs, as the algorithm minimizes the objective)
			loss = (-l_clip + c_1*l_vf - c_2*dist_entropy).mean()

			print(loss)

			# Perform the optimization step
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()

In [95]:
s = torch.from_numpy(np.random.random((10, 9))).float()
a = torch.from_numpy((np.random.random((10, 1))*9).astype(int)).float()
log = torch.from_numpy(np.random.random((10))).float()
adv = torch.from_numpy(np.random.random(10)).float()
pvv = torch.from_numpy(np.random.random(10)).float()
print(a.size(0))
ppo_step(10, 3, s, a, log, adv, pvv)

10
tensor(0.2747, grad_fn=<MeanBackward0>)
tensor(0.0628, grad_fn=<MeanBackward0>)
tensor(0.1270, grad_fn=<MeanBackward0>)
tensor(0.0916, grad_fn=<MeanBackward0>)
tensor(0.0328, grad_fn=<MeanBackward0>)
tensor(0.2511, grad_fn=<MeanBackward0>)
tensor(0.0741, grad_fn=<MeanBackward0>)
tensor(0.2710, grad_fn=<MeanBackward0>)
tensor(0.0415, grad_fn=<MeanBackward0>)
